<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Query using TAP**

The ESO TAP service allows you to issue custom ADQL (Astronomical Data Query Language) queries against the archive metadata, offering fine-grained control over your search. TAP queries can be issued against different tables, depending on the type of data you’re interested in:

* The `ivoa.ObsCore` table provides standardized metadata for fully calibrated (Phase 3) data products.
* The `dbo.raw` table provides access to raw observational data across all ESO instruments.
* The `ist.<instrument_name>` tables (e.g. `ist.midi`, `ist.muse`) allow more detailed queries tailored to instrument-specific raw metadata.

<hr style="border:2px solid #0281c9"> </hr>

# **Importing and basic usage of astroquery.eso**

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10290


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

# **Query the ESO Archive using the TAP Service**

## **Reduced data search (ivoa.ObsCore)**

This example demonstrates how to use `astroquery.eso` to search the `VEGAS` (VST Early-type GAlaxieS Survey) collection for fully reduced, science-ready imaging data of bright galaxies from the NGC catalog. By filtering for Phase 3 calibrated products with fine spatial resolution, this query retrieves recent deep optical images well-suited for studying the structure and low-surface-brightness features of nearby early-type galaxies.

In [5]:
adql_query = """
            SELECT TOP 30 target_name, s_ra, s_dec, instrument_name, 
                obs_collection, dataproduct_type, s_pixel_scale, 
                calib_level, t_min, proposal_id, dp_id
            FROM ivoa.ObsCore
            WHERE obs_collection = 'vegas'
                AND calib_level = 3
                AND dataproduct_type = 'image'
                AND s_pixel_scale < 0.4
                AND target_name LIKE 'NGC%'
            ORDER BY t_min DESC
            """

table = eso.query_tap_service(adql_query)
table["target_name", "s_ra", "s_dec", "instrument_name", "dataproduct_type", "s_pixel_scale", "dp_id"][:5]

target_name,s_ra,s_dec,instrument_name,dataproduct_type,s_pixel_scale,dp_id
,deg,deg,,,arcsec,
object,float64,float64,object,object,float64,object
ngc1533,62.389416,-56.114055,OMEGACAM,image,0.21,ADP.2021-02-01T14:04:33.669
ngc1533,62.444166,-56.100777,OMEGACAM,image,0.21,ADP.2021-02-01T14:04:33.667
ngc5018,197.876458,-19.336277,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.675
ngc3379,161.513666,12.642055,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.679
ngc5018,197.876458,-19.336277,OMEGACAM,image,0.2099,ADP.2021-02-01T14:04:33.673


## **Raw data search**

This example demonstrates how to use `astroquery.eso` to search for **raw narrow-band imaging data** around the Hα line (656.3 nm) for the galaxy **NGC 3627**. The query filters for science exposures within a 0.1-degree radius of the galaxy's coordinates, selects only observations with a narrow spectral band that includes Hα, and excludes broadband filters by requiring the filter width to be less than 20 nm. This approach is useful for identifying candidate raw frames suitable for star formation or ionized gas studies in nearby galaxies. The result returns key metadata such as object name, position, instrument, exposure time, and access URLs for download.

In [ ]:
adql_query = """
            SELECT *
            FROM dbo.raw
            WHERE dp_cat = 'SCIENCE'
                AND 1 = INTERSECTS(s_region, CIRCLE('', 170.0626, 12.9915, 0.1))  -- defining a cone of 0.1 deg radius around NGC 3627
                AND lambda_min < 656 
                AND lambda_max > 656                                -- Ensure Hα falls in the filter bandpass
                AND lambda_max - lambda_min < 20                    -- Narrow-band filter (~< 20 nm)
            ORDER BY mjd_obs
            """

table = eso.query_tap_service(adql_query)
table["object", "ra", "dec", "instrument", "exposure", "tel_ambi_fwhm_start", "date_obs", "prog_id", "dp_id", "access_url"][:5]

object,ra,dec,instrument,exposure,tel_ambi_fwhm_start,date_obs,prog_id,dp_id,access_url
,deg,deg,,s,arcsec,,,,
object,float64,float64,object,float32,float32,object,object,object,object
NGC-3627,170.06142694,12.99157,WFI,89.917,0.68,2019-02-08T07:03:09.793,0102.A-9002(A),WFI.2019-02-08T07:03:09.793,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:03:09.793
NGC-3627,169.97285888,12.95482,WFI,89.917,0.68,2019-02-08T07:12:44.346,0102.A-9002(A),WFI.2019-02-08T07:12:44.347,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:12:44.347
NGC-3627,169.98158694,12.96159,WFI,89.917,0.68,2019-02-08T07:15:41.075,0102.A-9002(A),WFI.2019-02-08T07:15:41.076,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:15:41.076
NGC-3627,169.97238111,12.955,WFI,359.917,0.68,2019-02-08T07:18:33.565,0102.A-9002(A),WFI.2019-02-08T07:18:33.565,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:18:33.565
NGC-3627,169.98128888,12.96175,WFI,359.917,0.68,2019-02-08T07:25:59.174,0102.A-9002(A),WFI.2019-02-08T07:25:59.174,https://dataportal.eso.org/dataPortal/file/WFI.2019-02-08T07:25:59.174


It is always useful to check which columns are available when querying instrument-specific tables, as they differ from `dbo.raw` and can vary between instruments (i.e. differnt `ist.` tables). This can be done within astroquery as follows. 

**Note:** For more information about the TAP and how to write ADQL queries, refer to the following resources:

* ESO TAP+ documentation: Describes ESO’s implementation of TAP and the available services.
* IVOA TAP standard: The official specification from the International Virtual Observatory Alliance.
* ADQL specification: Defines the query language used to interact with TAP service

<hr style="border:2px solid #0281c9"> </hr>